Knowledge Base Agents and Reliability
Building effective AI agents requires expanding their knowledge sources and ensuring they function reliably in real-world applications. Whether developing a customer support bot or a complex AI assistant, agents need access to accurate data and well-structured workflows to improve their performance.

Expanding an Agent’s Knowledge
Agents can go beyond simple LLM-powered responses by integrating external data sources.

APIs – Provide real-time data, allowing agents to access updated information.
Long-Term Memory – Enables agents to store and recall past interactions by saving data in a database.
Retrieval-Augmented Generation (RAG) – Enhances agents by retrieving relevant knowledge before generating a response.
Designing for Reliability
Reliability means ensuring an agent performs its intended function efficiently and safely. Several factors must be considered:

Defining the Agent’s Role – Should an agent handle multiple tasks, or should there be specialized agents with clear objectives?
Measuring Performance – Is the agent efficient? If a task takes 10 minutes and thousands of tokens, it may need optimization.
Assessing Success Probability – Establish evaluation criteria before deploying an agent.
Understanding the Operational Environment – A banking agent assisting investors may require a different workflow than one advising loan applicants.
Preventing Harm – Restrict agent permissions to avoid critical failures, such as deleting an entire database table.
Techniques for Improving Reliability
Human-in-the-Loop – In critical cases, human oversight ensures correctness.
Observability – Tracking response time, accuracy, and user interactions helps in optimizing agent behavior.
Evaluation – Unlike traditional ML models, agentic systems require continuous, iterative evaluation to maintain quality.
Final Thoughts
AI applications must be designed for reliability by carefully structuring workflows, integrating external knowledge sources, and implementing best practices. In the next lesson, these concepts will be applied using LangGraph.

Enhancing an Agent’s Knowledge
An agent’s knowledge can be improved by optimizing its internal components. These can be grouped into three categories:

Understanding – How well the underlying model interprets and reasons about user inputs.
Context – Additional instructions or external data provided to shape the agent’s responses.
Memory – Storage and retrieval of past interactions to ensure continuity across conversations.
Understanding
An LLM processes language by predicting the most probable next token, enabling it to interpret inputs and adapt to different situations.

Ways to improve understanding:

Use a larger model with more training data and better reasoning capabilities.
Fine-tune a model with high-quality, domain-specific data to enhance performance in a particular field.
A strong underlying model ensures the agent can handle diverse queries and make informed decisions.

Context
Context enhances an agent’s decision-making by providing background information and external tools.

Ways of adding context:

System Prompts – Provide procedural guidelines.
Few-Shot Prompting – Supplies examples of how to respond to specific inputs, improving behavior.
Tool Use – Allows the model to select and call external functions when needed.
Knowledge Bases (RAG) – Uses Retrieval-Augmented Generation to fetch relevant documents, reducing the need for complex prompts.
Memory
Since LLMs are stateless, they do not remember past interactions unless memory is explicitly managed.

Types of memory storage:

Short-Term Memory – Maintains conversation continuity within a single interaction loop but resets once the loop ends.

In-Session Memory – Stores conversation history for the duration of a session, allowing multi-turn interactions.

Across-Session Memory – Retains long-term knowledge of past user interactions, preferences, or previous agent actions across multiple sessions.

Balancing Knowledge and Cost
While adding context and memory improves agent performance, it comes with trade-offs:

Stateless models require full context for every invocation.
Token limits restrict how much information can be included in a prompt.
Larger models and longer prompts increase computational costs.
Effective AI workflow design requires balancing context, memory, and efficiency to create knowledgeable and scalable agents.

Summary: Calling External APIs with Tools in LangGraph Workflows
Overview
This demo explains how to integrate external API calls into LangGraph workflows by building custom tools. It highlights the use of real-time data retrieval (quotes and web search) and shows how agents interact with external information sources.

In [2]:
import os
from typing import Dict
import requests
from tavily import TavilyClient
from langchain_core.tools import tool
from langchain_core.messages import (
    SystemMessage,
    AIMessage,
    HumanMessage, 
    ToolMessage,
)
from langchain_openai import ChatOpenAI
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import MessagesState, add_messages
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from dotenv import load_dotenv

In [3]:
load_dotenv()

False

In [4]:
@tool
# won't work within ACN - website restricted
def random_got_quote_tool()->Dict:
    """
    Return a random Game of Thrones quote and the character who said it
    """
    response = requests.get("https://api.gameofthronesquotes.xyz/v1/random")
    return response.json()

In [5]:
random_got_quote_tool.invoke({})

SSLError: HTTPSConnectionPool(host='api.gameofthronesquotes.xyz', port=443): Max retries exceeded with url: /v1/random (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1032)')))

In [ ]:
tavily_client = TavilyClient(
    api_key=os.getenv("TAVILY_API_KEY")
)

In [ ]:
@tool
def web_search(question:str)->Dict:
    """
    Return top search results for a given search query
    """
    response = tavily_client.search(question)
    return response

In [ ]:
web_search.invoke(
    {
        "question": "Who performs Cersei Lannister in Game of Thrones?"
    }
)

In [ ]:
tools = [random_got_quote_tool, web_search]

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def agent(state: MessagesState):
    ai_message = llm_with_tools.invoke(state["messages"])
    return {"messages": ai_message}

In [ ]:
def router(state: MessagesState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"

    return END

In [ ]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", agent)
workflow.add_node("tools", ToolNode(tools))

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    source="agent", 
    path=router, 
    path_map=["tools", END]
)

workflow.add_edge("tools", "agent")


In [ ]:
graph = workflow.compile()

display(
    Image(
        graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
messages = [
    SystemMessage(
        "You are a Web Researcher focused on Game of Thrones. "
        "If user asks you a random quote about GoT. You will not only " 
        "provide it, but also search the web to find the actor or actress "
        "who perform the character who said that."
        "So, your output should be: Quote, Character and Performer."
    ),
    HumanMessage("Give me a radom GoT quote")
]

In [ ]:
result = graph.invoke(
    input={
        "messages": messages
    }
)

print(result)

In [ ]:
for message in result["messages"]:
    message.pretty_print()

Key Steps Covered
1. Setting up External API Tools
a. Random Game of Thrones Quote Tool
A tool is created to fetch a random Game of Thrones quote.

Uses a public API endpoint (https://api.gameofthronesquotes.xyz/v1/random).


@tool

def random_got_quote():

  response = requests.get("https://api.gameofthronesquotes.xyz/v1/random")

  return response.json()
Sample outputs include:

"sentence": "The things I do for love."

"character": "Jaime Lannister"

b. Tavily Web Search Tool
A second tool is built for live web search using the Tavily API.

Allows answering follow-up questions based on current information.


@tool

def web_search(question: str):

  response = tavily_client.search(query=question)

  return response
Example use:

Question: "Who performs Cersei Lannister in Game of Thrones?"

Top result: "Lena Headey" from Wikipedia.

2. Binding Tools to the LLM
The tools are bound to the LLM using bind_tools.

This creates an LLM with Tools object.


llm_with_tools = llm.bind_tools([random_got_quote, web_search])
An agent abstraction is built around the LLM with tools.
3. Router Logic
A router determines if tool usage is necessary:

If the last message includes a tool call, the workflow routes to the tools node.

Otherwise, it terminates.


def router(state):

  last_message = state.messages[-1]

  if last_message.tool_calls:

      return "tools"

  return "end"
4. Workflow Construction
A LangGraph StateGraph is set up using MessageState to manage conversation history.

Nodes:

agent node: Handles standard LLM interaction.

tools node: Executes the external API tool calls.

Edges:

start → agent

agent → tools (conditionally, if needed)

tools → agent (loop)

Terminate when no tool calls are made.


workflow.add_node("agent", agent_node)

workflow.add_node("tools", tools_node)

workflow.add_edge("start", "agent")

workflow.add_conditional_edges("agent", router)

workflow.add_edge("tools", "agent")
5. Execution Example
System sets the agent’s personality: "You are a web researcher focused on Game of Thrones."

Human asks: "Give me a random Game of Thrones quote."

Flow:

Agent calls random_got_quote.

Receives a quote (e.g., by Jaime Lannister).

Agent decides it needs to find the actor.

Calls web_search for "Jaime Lannister actor."

Receives result: "Nikolaj Coster-Waldau."

Outputs:

Sentence

Character

Actor

Associated URLs for more information.

6. Key Concepts Highlighted
Tool integration enables real-time, dynamic retrieval of external data.

Routers allow conditional control flow inside the workflow.

MessageState tracks conversation memory between user, LLM, and tool outputs.

Multiple tools can be chained flexibly based on LLM reasoning.

7. Conclusion
External APIs extend an agent's knowledge and capability far beyond static LLM training.

Combining API tools, LLM reasoning, and structured workflows creates powerful, responsive systems.

LangGraph provides a clean, modular architecture to manage this complexity.

## Next demo - persisting memory with a database in LangGraph

Overview  
This demo explains how to persist memory across sessions by saving conversation history into a SQLite database instead of keeping it only in RAM. This makes it possible to resume conversations later and builds the foundation for session-based systems.
  
Key Steps Covered  
1. Helper Function for Running Graphs  
A run_graph helper function is created to simplify invoking the graph repeatedly:

Takes a query, the graph object, and a thread_id.

def run_graph(query, graph, thread_id):

  ...


In [ ]:
import sqlite3
import json
from langchain_core.messages import (
    SystemMessage,
    AIMessage,
    HumanMessage, 
    ToolMessage,
)
from langchain_openai import ChatOpenAI
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from IPython.display import Image, display

In [ ]:
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

In [ ]:
def run_graph(query:str, graph:StateGraph, thread_id:int):
    output = graph.invoke(
        config={"configurable":{"thread_id": thread_id}},
        input={"messages":[HumanMessage(query)]}
    )
    return output

2. In-Memory Workflow Setup  
a. Workflow Definition  
A simple workflow with a single chatbot node.
  
State: Based on MessageState.
  

workflow = StateGraph(MessageState)

workflow.add_node("chatbot", chatbot_node)

workflow.add_edge("start", "chatbot")

workflow.add_edge("chatbot", "end")  
b. MemorySaver Checkpointer  
A MemorySaver is used to checkpoint states in RAM only.

from langgraph.checkpoints import MemorySaver

memory = MemorySaver()

workflow = StateGraph(MessageState, checkpointer=memory)  
c. Execution  
Queries like "What is memory?" are sent to the chatbot.

Metadata about each step (node traversed, messages) is collected internally in memory.

In [ ]:
def chatbot(state: MessagesState):
    ai_message = llm.invoke(state["messages"])
    return {"messages": ai_message}


workflow = StateGraph(MessagesState)

workflow.add_node(chatbot)
workflow.add_edge(START, "chatbot")
workflow.add_edge("chatbot", END)

checkpointer = MemorySaver()
in_memory_graph = workflow.compile(checkpointer=checkpointer)

display(
    Image(
        in_memory_graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
run_graph(
    query="Hi",
    graph=in_memory_graph, 
    thread_id="1"
)

In [ ]:
list(
    in_memory_graph.get_state(
        config={"configurable":{"thread_id": "1"}}
    )
)

3. SQLite-Persisted Workflow Setup  
a. SqliteSaver Checkpointer  
A SqliteSaver is used to persist workflow state to a SQLite database file (memory.db).

from langgraph.checkpoints import SqliteSaver

memory = SqliteSaver(db_path="memory.db")

workflow = StateGraph(MessageState, checkpointer=memory)
The database file can be accessed and queried independently.  
b. Execution  
The same queries are sent, but this time, metadata and snapshots are saved into the SQLite database.

In [ ]:
# For production, use something like Postgres
db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)

In [ ]:
memory = SqliteSaver(conn)
external_memory_graph = workflow.compile(checkpointer=memory)

display(
    Image(
        external_memory_graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
run_graph(
    query="What's a memory?",
    graph=external_memory_graph, 
    thread_id="2"
)

In [ ]:
list(
    external_memory_graph.get_state(
        config={"configurable":{"thread_id": "2"}}
    )
)

4. Inspecting the SQLite Database  
a. Schema Inspection  
A cursor is created to query the database.

Two tables are found:

checkpoints

writes


SELECT name FROM sqlite_master WHERE type='table'  
b. Metadata Retrieval  
Metadata columns contain serialized snapshots of:

Node transitions

Message exchanges (e.g., HumanMessage, AIMessage)

Model configurations


SELECT metadata FROM checkpoints  
Example metadata entries:  

Step -1: HumanMessage ("What is memory?")
  
Step 0: System setup
  
Step 1: AIMessage ("Memory can refer to several concepts...")
  
c. Advantages  
Full history of each thread is preserved.

Metadata includes model names, message types, and conversation content.

Conversations can be resumed at any point by using the same thread_id.

In [ ]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table_name[0] for table_name in cursor.fetchall()]
tables

In [ ]:
columns_map = []
results = []

for table in tables:
    cursor.execute(f"select * from {table}")
    results.append(cursor.fetchall())
    columns_map.append({table:[desc[0] for desc in cursor.description]})

In [ ]:
columns_map

In [ ]:
cursor.execute(f"select metadata from checkpoints")
metadata = cursor.fetchall()
metadata

In [ ]:
steps = [json.loads(m[0]) for m in metadata]
steps

In [ ]:
5. Key Concepts Highlighted  
MemorySaver is temporary; disappears when the session ends.

SqliteSaver creates persistent, resumable sessions.

Thread IDs differentiate multiple parallel conversations.

Session management becomes trivial with database persistence:

Can resume, search, or audit conversations.  
6. Conclusion  
Persisting memory with a database turns ephemeral conversations into durable sessions.

LangGraph's checkpoint system allows flexible storage backends.

This pattern is crucial for production-grade chatbot and agent applications that require history continuity across sessions.

## RAG Pipelines: Enhancing AI Agents with Retrieval and Generation

Retrieval-Augmented Generation (RAG) enhances AI agents by retrieving relevant data from external sources and generating informed responses based on that data. This technique improves accuracy, ensures up-to-date information, and provides contextually relevant answers that go beyond an LLM’s training data.

How a RAG Pipeline Works
Retrieval
The user submits a query.
The query is converted into a vector representation using an embedding model.
A vector database searches for similar stored content based on semantic similarity.
Augmentation
The retrieved information is added to the model’s context window.
This enriches the input so the model has both the query and relevant background knowledge.
Generation
The LLM processes the augmented input and generates a response.
The final output combines the model’s internal knowledge with retrieved external information.
Use Case: E-Commerce Customer Support
A user asks: “What is the return policy for electronics?”

Without RAG – The agent gives a generic response based on its training data, which may be outdated or inaccurate.

With RAG – The agent retrieves the actual return policy document, finds the relevant section, and generates a response based on company guidelines.

Preparing Data for RAG Pipelines
Before retrieval and generation can occur, documents must be collected, processed, and stored efficiently.

Data Collection
Sources include PDFs, websites, internal databases, and FAQs.
Preprocessing
Cleaning: Remove unnecessary text like HTML tags or special characters.
Chunking: Break large documents into smaller sections (e.g., paragraphs or sentences).
Embedding Generation: Convert text chunks into vector representations using models like OpenAI embeddings or bge-m3.
Storage
Store embeddings in a vector database (e.g., ChromaDB).
Include metadata (e.g., source, date) to enable efficient filtering.
RAG vs. Fine-Tuning
Both RAG and fine-tuning enhance an agent’s performance but in different ways:

Aspect	RAG	Fine-Tuning
How it works	Retrieves external data at runtime	Adjusts model weights using new data
Best for	Dynamic, frequently updated data	Domain-specific, long-term improvements
Data needs	Large document corpus for retrieval	Small, high-quality training dataset
Cost	Requires vector storage, minimal compute	Requires computational resources
Risk	Retrieval quality impacts accuracy	Catastrophic forgetting is possible
Choosing Between RAG and Fine-Tuning
The best approach is combining both:

Start with an LLM and strategic prompting – Quickly test feasibility.

Integrate RAG – Improve accuracy with external data retrieval.

Fine-tune a smaller model – Optimize performance once stable, reducing cost and latency.

A fine-tuned customer support agent using RAG for live knowledge retrieval creates a personalized, real-time experience that is accurate and cost-efficient.

Both techniques are powerful in agent design, and the right balance depends on the use case, cost, and update frequency of the required knowledge.